In [3]:
from casadi import *

**Problem**:

Wir wollen den eintretenden Strom in den Server begrenzen, **wenn nötig** und dem sendenden Server mitteilen, dass noch mehr geschickt werden kann, **wenn möglich**.
- $v_{in,req}$: gesendeter Strom, kann den Server überlasten und muss deshalb abgeschnitten werden, wenn:
- $v_{in,req}\geq v_{in,max}$, dabei ist $v_{in,max}$ eine Stellgröße.
- $v_{in}$: tatsächlich eintretender Strom

Wir wollen aber auch gleichzeitig erreichen, dass $v_{in,max}$ möglichst groß gewählt wird, wenn $v_{in,req}\leq v_{in,max}$. 


**Lösungsansatz:**

Hilfsvariablen einführen, sodass:
\begin{gather}
v_{in}=v_{in,req}-v_{in,discard}\\
v_{in,max}=v_{in}+v_{in,extra}
\end{gather}
Es soll aber nur entweder verworfen oder hinzugefügt werden. Deshalb setzten wir eine Beschränkung:
\begin{gather}
v_{in,discard}\cdot v_{in,extra} = 0
\end{gather}
Um das Ziel zu erreichen wird die Kostenfunktion zu:
\begin{gather}
obj = v_{in,discard}-v_{in,extra}
\end{gather}
gewählt. 

In [38]:
v_req = SX.sym('v_req',1,1)
v_in_extra = SX.sym('v_in_extra',1,1)
v_in_discard = SX.sym('v_in_discard',1,1)
v_max = SX.sym('v_max',1,1)

v_in = v_req-v_in_discard
v_in_max = v_in+v_in_extra

obj = -v_in_extra+v_in_discard

cons = vertcat(*[
    -v_in_extra,
    -v_in_discard,
    v_in-v_max,
    v_in_max-v_max
])

eq_cons = v_in_extra*v_in_discard
# Bounds for the constraints:
ub = np.zeros((5,1))
lb = -np.inf*np.ones((5,1))

# For the last constraint we set:
eps = 1e-6
lb[-1] =-eps
ub[-1] =eps

optim_dict = {
    'x': vertcat(v_in_extra,v_in_discard),
    'f': obj,
    'g': vertcat(cons, eq_cons),
    'p': vertcat(v_req, v_max)
}

In [39]:
opts = {'ipopt.linear_solver': 'MA27'}
S = nlpsol('S', 'ipopt', optim_dict, opts)

In [40]:
sol=S(ubg=ub,lbg=lb,p=np.array([10,5]), x0=np.array([0,0]))
sol

This is Ipopt version 3.12.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        7
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        1
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 5.00e+00 6.36e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   

{'f': DM(5),
 'g': DM([-1.15089e-07, -5, -1.09942e-07, 0, 5.75444e-07]),
 'lam_g': DM([0.0200409, 5.01156e-10, 0.00117168, 0.998828, 0.00424251]),
 'lam_p': DM([-1, 1]),
 'lam_x': DM([0, 0]),
 'x': DM([1.15089e-07, 5])}

In [37]:
sol_x = sol['x'].full()
print('v_in_extra = {0} and v_in_discard = {1}'.format(sol_x[0], sol_x[1]))

v_in_extra = [1.15088891e-07] and v_in_discard = [5.00000011]


**Note:** Unfortunately this formulation leads to the effect that the optimal solution is only found for extremely low values of the primal infeasibility. Otherwise the dual infeasibility is not achieved.